*   !pip install : used as package installer for python
*   -q : its "quiet" mode that means it will shows less output during installation proc
*   accelerate == 0.21.0 : optimizes and accelerates training of deep learning models
*   peft == 0.4.0 : for efficient fine tuning of LLMs
*   bitsandbytes -- 0.4.0 : provides optimized numerical operations for deep learning
*   tranformers == 4.31.0 : by hugging face that has pre trained models for NLP tasks
*   trl == 0.4.7 : stands for transformer reinforcement learning and used for fine tuning LLM using reinforcement learning techniques
*   ~= operator is compatible release to install latest pip version and compatible w/ specified versions























In [1]:
!pip install -q accelerate~=0.21.0 peft~=0.4.0 bitsandbytes~=0.40.2 transformers~=4.31.0 trl~=0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00


*   import os : functions to interact w/ OS i.e. python script can talk to the sys
*   import torch : pytorch by facebook. help build and train neural networks
*   from datasets import load_dataset : by hugging face and used to easily download and prepare datasets to train and test
*   AutoModelForCausalLM : class that automatically loads pre-trained language models that trained on text dataset to generate coherent natural sounding language
*   AutoTokenizer : represents tokenizer that converts text into format that model can understand and smaller parts called tokens.
*   BitsAndBytesConfig : for efficient compression and decompression of data for model storage and inference.
*   HfArgumentParser : parser for command-line arguments specifically designed to work with Hugging face ecosystem and allows users to pass options to script when running from cmd
*   TrainingArguments : contains arguments related to training process like learning rate, batch size, no. of epochs, etc
*   pipeline : chains multiple NLP tasks like text generation/classification, etc to process data in pipeline fashion
*   logging : this logs info, warnings and errors during model training, debugging and evaluation process
*   LoraConfig : stands for Low-Rank Adaptation that helps fine-tune pretrained models w/ fewer parameters and help them adapt to new tasks and datasets
*   PeftModel : Parameter-Efficient Fine-Tuning is type of AI model to be efficient and adaptable
*   SFTTrainer : class used for supervised fine-tuning (SFT) of lang models and simplifies process of training and tuning models on specific tasks or datasets


In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

*   NousResearch/Llama-2-7b-chat-hf : the chatbot model that we will be using.
*   mlabonne/guanaco-llama2-1k : this is the dataset that we will use to train and test the model
*   llama-2-7b-miniguanaco : the new name you are going to give to the fine tuned model

> syntax for using hugging face:
*   username/model_name
*   author/dataset_name
*   we find out the model card of the model you want to use and then write it in this syntax to use it





In [3]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
new_model = "llama-2-7b-miniguanaco"

### QLoRA parameters
*   lora_r = 64 : sets rank of low-rank matrices used. higher rank generally means more expressiveness but more computational cost. 64 is standard. it helps the model reduce the complexity of understanding the data. it basically determines the size
*   lora_alpha = 16 : this determines how strongly should it influence. it scales the output of LoRA layers before they are combined with w/ models original output. it balances contribution of og model and the changes by LoRA
*   lora_dropout = 0.1 : probability of randomly setting neuron's output to zero during training. it helps model prevent model from memorizing training data amd encourages it to learn more robust patterns.



In [4]:
# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

### bitsandbytes parameters
*   use_4bit = True : choose a specific small bit of data to load so its faster and easier rather than using any large amount. compressing the brain of the model but not reducing its intelligence
*   bnb_4bit_compute_dtype = "float16" : need the 4 bit computation to be calculated in only float 16 data type. creating balance between speed and precision
*   bnb_4bit_quant_type = "nf4" : setting quantization to be performed using 4 bit integers. this quantization works best for models like Llama 2 cause doesnt distort its knowledge too much
*   use_nested_quant = False : nested quantization means the model will be made more smaller and concise to be accurate although we are not doing it. if we quantize more then it could damage the performance.
*   amp_enabled = True : allows you to perform computations in a lower precision (e.g., FP16) while maintaining FP32 precision for critical parts of the model. This can help reduce memory usage and improve training speed.
*   gradient_accumulation_steps = 2 : If memory is a constraint, use gradient accumulation to effectively increase the batch size without requiring more GPU memory. This can improve the stability and efficiency of fine-tuning











In [5]:
# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Enable AMP (Automatic Mixed Precision)
amp_enabled = True

# Gradient accumulation to increase effective batch size
gradient_accumulation_steps = 2

### TrainingArguments parameters
*   output_dir = "./results" : dir where model's predictions and states will be stored
*   num_train_epochs = 4 : no. of times entire training data will be passed thru the model
*   fp16 = True, bf16 = False : they control whether use 16 or 32 bit floating pt precision for computations
*   per_device_train_batch_size = 8, per_devce_eval_batch_size = 8 : set bactch size per GPU for training and evaluating and take 8 eg at a time
*   gradient_accumulation_steps = 1 : number of update steps to accumulate gradients before performing a single optimization steps. Setting it to 1 means the gradients are not accumulated and are used immediately for optimization.
*   gradient_checkpointing = True : we enable gradient checkpointing. technique to reduce memory storage during training and keep checking if anything goes wrong or not
*   max_grad_norm = 0.3 : Gradient clipping is applied to prevent the gradients from becoming too large, which can cause instability in training
*   learning_rate = 2e-4 : sets the initial learning rate for the AdamW optimizer to 0.0002. The learning rate determines the step size at which the model's parameters are updated during training.
*   weight_decay = 0.001 : Weight decay is a regularization technique that adds a penalty term to the loss function, encouraging the model to have smaller parameter values and prevent overfitting.
*   optim = "paged_adamw_32bit" : variant of the AdamW optimizer that uses 32-bit precision and paged memory for efficient memory usage.
*   lr_scheduler_type = "cosine" : learning rate scheduler determines how the learning rate changes over the course of training. A cosine scheduler gradually decreases the learning rate following a cosine curve.
*   max_steps = -1 : If set to -1, the number of training steps will be determined based on the num_train_epochs and the size of the training dataset.
*   warmup_ratio = 0.03 : learning rate gradually increases from 0 to the initial learning rate over a specified fraction of the total training steps. In this case, 3% of the training steps will be used for warmup.
*   group_by_length = True : enables grouping sequences into batches with the same length. This can save memory and speed up training by avoiding padding and allowing more efficient computation.
*   save_steps = 0 : sets the number of update steps after which the model checkpoint is saved. If set to 0, the model will not be saved during training.
*   logging_steps = 25 : sets the logging frequency. The training progress and metrics will be logged every 25 update steps.





In [6]:
# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 4

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = True
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 8

# Batch size per GPU for evaluation
per_device_eval_batch_size = 8

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 2

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 500

# Log every X updates steps
logging_steps = 50


### SFT parameters
*   max_seq_length = None : it means that the model will use the maximum sequence length that it was pretrained with. Some models have a fixed limit on how much text they can process at once. Setting this to None means we're not imposing any such limit.
*   packing = False : Sequence packing is a technique where multiple short examples are packed into the same input sequence to increase efficiency.it means that each example will be processed independently, without any packing.
*   device_map = {"":0} : device mapping determines which parts of the model are loaded on which devices (GPUs). this means that the entire model will be loaded on GPU 0. empty string "" represents the entire model, and 0 represents the GPU index.
*   List item


In [7]:
# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = True

# Load the entire model on the GPU 0
device_map = {"": 0}

*   dataset = load_dataset(dataset_name, split = "train") : this is to load the dataset that we had initialized above. the dataset is then split into training dataset.
*   compute_dtype = getattr(torch, bnb_4bit_compute_dtype) : we're like choosing the right tools for the job. so here we choose the torch library that works perfectly for numbers and then we pass the datatype that defined above.
*   bnb_config = BitsAndBytesConfig(...) : define the QLoRA as we did above
*   load_in_4bit = use_4bit : shrinking the size of information so its easier for model to handle and it also saves space and processes information faster
*   bnb_4bit_quant_type = bnb_4bit_quant_type : here we define choosing the perfect quantization type to represent the information in the model more efficiently
*   bnb_4bit_compute_dtype = compute_dtype : makes sure the tools that are chosen have smaller and simpler instructions we're using for the tools we selected above
*   bnb_4bit_use_double_quant = use_nested_quant : makes it more concise and optimized




In [8]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

*   if compute_dtype == torch.float16 and use_4bit : checks if the specified compute data type is torch.float16 (half-precision floating-point) and if 4-bit quantization is being used.
*   major, _ = torch.cude.get_device_capability() : major value indicates the generation of your GPU architecture. _ is used as a placeholder for the minor version number. provides CUDA-related functionality, allowing PyTorch to interact with NVIDIA GPUs. If CUDA is working, the function returns a tuple of two numbers: (major, minor). The purpose of retrieving the CUDA compute capability in this code is to check if the GPU supports the bfloat16 data type.
*   if major >= 8 : GPUs with a compute capability of 8.0 or higher have native support for bfloat16, which can accelerate training by using this lower-precision data type
*   print stmts in the if : if your GPU supports bfloat16, this code will suggest enabling it (bf16=True) for faster training.
*   model = AutoModelForCausalLM.from_pretrained(...) : loads the pre-trained Llama 2 model from Hugging Face like the version of model we mentioned in above code
*   model_name : the model of Llama 2 that we defined above
*   quantization_config = bnb_config : Passes the quantization configuration (bnb_config) to the model loading function. This configuration specifies the settings for quantization, such as using 4-bit precision and the compute data type.
*   device_map = "auto": Specifies the device map for distributing the model across multiple devices (e.g., GPUs). automatically optimizes the placement for you.
*   model.config.use_cache = False : disables the attention cache, which can sometimes cause issues during fine-tuning, especially with QLoRA.
*   model.config.pretraining_tp = 1 : Sets the tensor parallelism attribute of the model configuration to 1. This attribute is specific to the Llama 2 model and used for controlling the level of parallelism during fine-tuning.



In [9]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

*   tokenizer = AutoTokenizer.from_pretrained(...) : loads the tokenizer for the LLaMA model using the AutoTokenizer class from the Hugging Face Transformers library. from_pretrained() method is used to load the tokenizer configuration from a pretrained model specified by model_name
*   trust_remote_code = True : set to allow loading code from remote sources, which is necessary for tokenizers.
*   tokenizer.pad_token = tokenizer.eos_token : sets the padding token of the tokenizer to be the same as the end-of-sequence (EOS) token.Padding tokens are used to pad sequences to a fixed length during batch processing.
By setting the padding token to the EOS token, the model will treat padded tokens as end-of-sequence markers.
*   tokenizer.padding_side = "right" : sets the padding side of the tokenizer to the right side of the sequences. if its set to default i.e. on the left then it can cause issues w/ fp16. so setting to right avoids potential overflow issues during training
*   peft_config = LoraConfig(...) : loads configurations for LoRA
*   lora_alpha = lora_alpha : sets value to the one we defined above
*   lora_dropout = lora_droupout : setting the value to the one we defined in above code
*   r = lora_r : sets value as defined above
*   bias = "none" : no biases are used
*   task_type = "CAUSAL_LM" :  type of task the model is being fine-tuned for i.e. set to "CAUSAL_LM", indicating a causal language modeling task where the model predicts the next token based on the previous tokens.




In [10]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

*   training_arguments = TrainingArguments(...) : creates class i.e. part of Transformers lib and used to define various parameters and configurations for training process.
*   output_dir = output_dir : it specifies the dir where trained model and other files are saved.
*   num_train_epochs = num_train_epochs : sets number of training epochs i.e. how many times the dataset will be passed through the model
*   per_device_train_batch_size = per_device_train_batch_size : batch size is number of samples that will be passed through the model at once during training. per_device is for specific device that you are using
*   gradient_accumulation_steps = gradient_accumulation_steps : Gradient accumulation is a technique used to effectively increase the batch size without running into memory limitations. It accumulates the gradients from multiple smaller batches before updating the model's parameters.
*   optim = optim : this parameter specifies the optimization algorithm to be used during training i.e. AdamW
*   save_steps = save_steps : requency at which to save the model's checkpoints during training
*   logging_steps = logging_steps : sets the frequency at which training logs will be generated, measured in number of steps.
*   learning_rate = learning_rate : controls how quickly the model learns from the data.
*   weight_decay = weight_decay : regularization technique to prevent overfitting by adding a penalty term to the loss function.
*   fp16 = fp16, bf16 = bf16 : enable the use of mixed precision training, which can improve training speed and efficiency by using lower-precision data types
*   max_grad_norm = max_grad_norm : sets the maximum allowed gradient norm, which is used to prevent the gradients from becoming too large during training.
*   max_steps = max_steps : sets the maximum number of training steps, which can be used as an alternative to specifying the number of training epochs
*   warmup_ratio = warmup_ratio : sets the ratio of the total training steps used for a linear warmup of the learning rate at the beginning of training
*   group_by_length = group_by_lenth : boolean indicating whether to group samples by their length during training  which can improve training efficiency and performance
*   lr_scheduler_type = lr_scheduler_type :  which can improve training efficiency and performance i.e. cosine
*   report_to = "tensorboard" : specifies that the training progress and metrics should be reported to TensorBoard, a popular tool for visualizing and monitoring machine learning experiments







In [11]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

*   trainer = SFTTrainer(..) : instance of the SFTTrainer class
*   model : the pre-trained model that we want to fine tune
*   train_dataset = dataset : the dataset that we had declared above
*   peft_config = peft_config : Parameter-Efficient Fine-Tuning (PEFT). PEFT is a technique that allows fine-tuning of LLMs with limited computational resources by only fine-tuning a subset of the model's parameters
*   dataset_text_field = "text" : name of the field in the dataset that contains the text data. In this case, it is set to "text"
*   max_seq_length = max_seq_length :  maximum sequence length for the input data. It determines the maximum number of tokens that the model will process at a time.
*   tokenizer = tokenizer : tokenizer object used to tokenize the input text data into a format that the model can understand
*   args = training_arguments : Additional training arguments specified in the training_arguments variable. These arguments may include hyperparameters, optimization settings, etc.
*   packing = packing : packing sequences for efficient training. It is likely used to optimize memory usage during training
*   trainer.train() : nitiates the training process using the train() method of the SFTTrainer instance. During training, the model will iterate over the provided training dataset, update its parameters based on the specified training arguments, and aim to minimize the training loss. The fine-tuning process will continue for the specified number of epochs
*   trainer.model.save_pretrained(new_model) : saves the fine-tuned model to a specified directory new_model. The save_pretrained() method is commonly used in Hugging Face's Transformers library to save the model's weights and configuration files in a format that can be easily loaded later for inference or further fine-tuning. new_model variable should contain the path or directory where you want to save the fine-tuned model. The saved model can then be loaded using the from_pretrained() method from the same library.



In [14]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

ValueError: the `--group_by_length` option is only available for `Dataset`, not `IterableDataset

*   logging.set_verbosity(logging.CRITICAL) : used to display messages and warnings during program execution. it tells the logging module to only display critical errors and suppress all other warnings and messages
*   prompt = "What is large language model?" : defines a variable called prompt and assigns it a string value, which represents the prompt or question that will be used as input for the text generation task
*   pipe = pipeline(..) : from the transformers library that creates a text generation pipeline
*   task = "text-generation" : Specifies that you want to use the pipeline for text generation.
*   model = model : set to the variable model, which should contain the loaded fine-tuned Llama 2 model.
*   tokenizer = tokenizer : set to the variable tokenizer, that contains the tokenizer associated with the fine-tuned Llama 2 model.
*   max_length = 200 : specifying the maximum length of the generated text in terms of the number of tokens.
*   result = pipe(f"< s >[INST] {prompt} [/INST]") : invokes the text generation pipeline (pipe) with a formatted input string. input string is constructed using an f-string, which allows embedding the prompt variable within the string. < s > represents the start of the sequence, [INST] and [/INST] are special tokens indicating the instruction or prompt, and {prompt} is replaced with the actual prompt value. then these values are saved in result variable
*   print(result[0]['generated_text']) : prints the generated text obtained from the text generation pipeline. result is expected to be a list containing a dictionary with the generated text.result[0] accesses the first (and likely only) dictionary in the list.['generated_text'] retrieves the value associated with the key 'generated_text' from the dictionary, which represents the generated text.
The generated text is then printed to the console.



In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "What is a large language model?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

*   del model : it deletes the model object i.e. the instance of Llama 2. we're removing the reference to the model's weights, buffers, and other associated data structures, which are stored in memory. This helps to free up memory, as the model is no longer occupying space.
*   del pipe : it deletes pipe object from memory. A pipeline is a sequence of processing steps that can include tokenization, encoding, and decoding. By deleting the pipeline, we're removing any remaining references to the model's internal state, which can also help to free up memory
*   del trainer : deletes the trainer object, holds references to the model, optimizer, and other training-related data structures, which are now deleted
*   import gc : imports garbage collector i.e. responsible for automatically freeing up memory that is no longer being used by the program
*   gc.collect() : trigger the garbage collection process to reclaim memory that is no longer needed. eason for calling gc.collect() twice is to ensure that any objects that were freed up in the first call and are no longer referenced by other objects are also collected and their memory is freed.


In [ ]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

*   base_model = AutoModelForCausalLM.from_pretrained(..) : loads a pre-trained LLaMA 2 model using the AutoModelForCausalLM class from the transformers library
*   model_name = name of the pre-trained model to load
*   low_cpu_mem_usage = True : this flag tells the model to use less CPU memory, which can be helpful when working with large models
*   return_dict = True : this flag indicates that the model should return a dictionary containing the output instead of a tuple.
*   torch_dtype = torch.float16 : this sets the data type of the model's weights and activations to float16 (FP16), which can reduce memory usage and improve performance on supported hardware.
*   device_map = device_map : this argument specifies a device map, which defines how the model's weights and buffers are distributed across multiple devices
*   model = PeftModel.from_pretrained(base_model, new_model) : creates a new PeftModel instance, which is a special type of model that allows for efficient fine-tuning of large language models using LoRA. base_model is pre-trained LLaMA 2 model loaded in the previous step. new_model is custom model or a configuration object that defines the LoRA weights and other modifications to the pre-trained model
*   model = model.merge_and_unload() : it merges the LoRA weights with the pre-trained model weights. Unloads the original pre-trained model weights to save memory
*   tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = True) : it loads a pre-trained tokenizer using the AutoTokenizer class from the transformers library. model_name is name of the pre-trained model to load. trust.. flag indicates that the tokenizer should trust remote code and execute it locally.
*   tokenizer.pad_token = tokenizer.eos_token : sets the pad_token attribute of the tokenizer to the eos_token (end-of-sequence token). This is done to ensure that the tokenizer uses the same token for padding and end-of-sequence marking.
*   tokenizer.padding_side = "right" : sets the padding_side attribute of the tokenizer to "right", indicating that padding should be applied to the right side of the input sequences.



In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

*   !huggingface-cli login : using the Hugging Face CLI (Command-Line Interface) to log in to your Hugging Face account. By running this command, you are authenticating yourself with the Hugging Face platform, which is necessary for the subsequent actions you'll be performing.
*   model.push_to_hub(new_model, use_temp_dir = False) : method provided by the Hugging Face library that allows you to upload your trained model to the Hugging Face Hub, which is a centralized repository for sharing and accessing machine learning models. new_model is the variable or object representing your fine-tuned LLaMA 2 model that you want to upload. use_temp_dir=False is a parameter that specifies whether to use a temporary directory to store the model files during the upload process. Setting it to False means that the model files will be uploaded directly from their original location, without creating a temporary directory.
*   tokenizer.push_to_hub(new_model, use_temp_dir = False) : pushing the tokenizer associated with your fine-tuned LLaMA 2 model to the Hugging Face Hub. tokenizer.push_to_hub() is a method provided by the Hugging Face library that allows you to upload the tokenizer, which is a crucial component for preprocessing text data for your model. new_model is the same variable or object representing your fine-tuned LLaMA 2 model. use_temp_dir=False is the same parameter as before, which specifies that the tokenizer files will be uploaded directly from their original location.


In [ ]:
!huggingface-cli login

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)